<a href="https://colab.research.google.com/github/Namesakenberg/Deep_Learning/blob/main/Complete_ANN_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [150]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [151]:
df = pd.read_csv('/content/diabetes.csv')

In [152]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [153]:
X = df.drop(columns=['Outcome'])
y = df['Outcome']

In [154]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X,y,random_state = 42 , test_size = 0.2)

In [155]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)

In [156]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout
from keras.callbacks import EarlyStopping

In [157]:
pip install keras_tuner

In [158]:
import keras_tuner as kt

In [159]:
def make_model(hp):
  model = Sequential()
  # hidden layers
  for i in range(hp.Int('num_layers' , min_value = 1 , max_value = 10)):

    units = hp.Int(f'units_{i}' , min_value=8 , max_value = 128)
    activation= hp.Choice(f'activation_{i}' , ['relu', 'sigmoid', 'tanh', 'elu', 'selu', 'swish'])

    if i == 0 :
      model.add(Dense(units =units , activation= activation,input_dim=X_train_scaled.shape[1]))
    else:
      model.add(Dense(units = units , activation= activation))

    model.add(Dropout(hp.Choice(f'dropout_{i}' , values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))

  model.add(Dense(units = 1 , activation= 'sigmoid'))

  model.compile(optimizer = hp.Choice('optimizer' , values = ['sgd', 'adam', 'adamax', 'nadam', 'adagrad', 'adadelta', 'rmsprop', 'ftrl'])
              , loss = 'binary_crossentropy'
              , metrics = ['accuracy'])

  return model

In [160]:
tuner = kt.RandomSearch(make_model , objective='val_accuracy' ,max_trials=5 , overwrite = True)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [161]:
tuner.search(X_train_scaled , y_train , epochs =  20, validation_data = (X_test_scaled , y_test))

Trial 5 Complete [00h 00m 12s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.7597402334213257
Total elapsed time: 00h 01m 12s


In [162]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 3,
 'units_0': 103,
 'activation_0': 'elu',
 'dropout_0': 0.4,
 'optimizer': 'rmsprop',
 'units_1': 35,
 'activation_1': 'selu',
 'dropout_1': 0.2,
 'units_2': 95,
 'activation_2': 'selu',
 'dropout_2': 0.7,
 'units_3': 39,
 'activation_3': 'sigmoid',
 'dropout_3': 0.7,
 'units_4': 46,
 'activation_4': 'relu',
 'dropout_4': 0.8,
 'units_5': 20,
 'activation_5': 'relu',
 'dropout_5': 0.2,
 'units_6': 114,
 'activation_6': 'sigmoid',
 'dropout_6': 0.2,
 'units_7': 100,
 'activation_7': 'tanh',
 'dropout_7': 0.5,
 'units_8': 97,
 'activation_8': 'elu',
 'dropout_8': 0.2}

In [163]:
model = tuner.get_best_models(num_models=1)[0]
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 103)            │           927 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 103)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 35)             │         3,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 35)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 95)             │         3,420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 95)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            96 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,083 (31.57 KB)

 Trainable params: 8,083 (31.57 KB)

 Non-trainable params: 0 (0.00 B)

In [164]:
'''
from keras.callbacks import EarlyStopping
callback = EarlyStopping(
    monitor='val_loss',
    min_delta=0.00001,
    patience= 20,
    verbose =1 ,
    mode= "auto",
    baseline= None,
    restore_best_weights=False
)
'''

'\nfrom keras.callbacks import EarlyStopping\ncallback = EarlyStopping(\n    monitor=\'val_loss\',\n    min_delta=0.00001,\n    patience= 20,\n    verbose =1 ,\n    mode= "auto",\n    baseline= None,\n    restore_best_weights=False\n)\n'

In [165]:
model.fit(X_train_scaled , y_train , epochs =100,validation_data=(X_test_scaled , y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - accuracy: 0.7332 - loss: 0.5520 - val_accuracy: 0.7403 - val_loss: 0.5110
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.7729 - loss: 0.4898 - val_accuracy: 0.7597 - val_loss: 0.5184
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.7593 - loss: 0.5390 - val_accuracy: 0.7273 - val_loss: 0.5300
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.7285 - loss: 0.5693 - val_accuracy: 0.7468 - val_loss: 0.5054
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.7274 - loss: 0.5483 - val_accuracy: 0.7532 - val_loss: 0.5003
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.7776 - loss: 0.4792 - val_accuracy: 0.7597 - val_loss: 0.5045
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.7504 - loss: 0.5365 - val_accuracy: 0.7403 - val_loss: 0.5259
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.7532 - loss: 0.5234 - val_accuracy: 0.

In [166]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
dct = DecisionTreeClassifier()
dct.fit(X_train , y_train)
y_pred = dct.predict(X_test)

accuracy_score(y_test , y_pred)

0.7337662337662337